In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

In [2]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [3]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim//num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attention_output = self.attention(inputs, inputs)
        self.attention_output_result = attention_output
        attention_output = self.dropout1(attention_output, training=training)
        out1 = self.layernorm1(inputs + attention_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        
        # residual connection
        output = self.layernorm2(out1 + ffn_output)
        return output

In [10]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen, padding='post')
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen, padding='post')

25000 Training sequences
25000 Validation sequences


In [11]:
word_index = keras.datasets.imdb.get_word_index()
index_word = dict((i, word) for (word, i) in word_index.items())

In [33]:
x_train.shape

(25000, 200)

In [14]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [12]:
x_train_emb = model.layers[1](x_train[:8, :])
x_train_tb = model.layers[2](x_train_emb)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddin  (None, 200, 32)          646400    
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 200, 32)          6464      
 merBlock)                                                       
                                                                 
 global_average_pooling1d (G  (None, 32)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_2 (Dropout)         (None, 32)                0     

In [30]:
model.layers[1].token_emb.metrics

[]

In [15]:
[l.shape for l in model.layers[1].weights]

[TensorShape([20000, 32]), TensorShape([200, 32])]

In [16]:
[l.shape for l in model.layers[2].attention.weights]

[TensorShape([32, 2, 16]),
 TensorShape([2, 16]),
 TensorShape([32, 2, 16]),
 TensorShape([2, 16]),
 TensorShape([32, 2, 16]),
 TensorShape([2, 16]),
 TensorShape([2, 16, 32]),
 TensorShape([32])]

# Multi-Head Attention

$\large Attention(Q,K,V) = softmax(\frac{QK^{T}}{\sqrt{d_k}})V$

[code source](https://github.com/keras-team/keras/blob/07e13740fd181fc3ddec7d9a594d8a08666645f6/keras/layers/attention/multi_head_attention.py)

## Prepare tensors: Query, Key, Value

In [18]:
att = model.layers[2].attention

query = att._query_dense(x_train_emb)

# `key` = [batch_size, max_len, num_heads, dim_hidden]
key = att._key_dense(x_train_emb)

# `value` = [batch_size, max_len, num_heads, dim_hidden]
value = att._value_dense(x_train_emb)

In [19]:
x_train_emb.shape

TensorShape([8, 200, 32])

In [20]:
query.shape

TensorShape([8, 200, 2, 16])

In [21]:
np.alltrue(tf.einsum('abc,cde', x_train_emb, att._query_dense.kernel)==query)

True

In [22]:
np.alltrue(tf.einsum('abc,cde', x_train_emb, att._key_dense.kernel)==key)

True

In [23]:
np.alltrue(tf.einsum('abc,cde', x_train_emb, att._value_dense.kernel)==value)

True

## Compute attention score and output

In [168]:
attention_mask = None
training = None


attention_output, attention_scores = att._compute_attention(
    query, key, value, attention_mask, training)
att_output_ = attention_output
attention_output = att._output_dense(attention_output)

In [169]:
import math
query = tf.multiply(query, 1.0 / math.sqrt(float(16)))

print('key shape:', key.shape)
print('query shape:', query.shape)
print('scaled dot product(key*query):', att._dot_product_equation)
att_score = tf.einsum(att._dot_product_equation, key, query)

att_score = att._masked_softmax(att_score, attention_mask)
print('attention score shape:',att_score.shape)

attention_scores_dropout = att._dropout_layer(att_score, training=training)

print()
print('value shape:', value.shape)
print('context vector(score*value):', att._combine_equation)
att_output = tf.einsum(att._combine_equation, attention_scores_dropout, value)
print('context vector shape:',att_output.shape)

key shape (8, 200, 2, 16)
query shape (8, 200, 2, 16)
scaled dot product(key*query): aecd,abcd->acbe
attention score shape: (8, 2, 200, 200)

context vector(score*value): acbe,aecd->abcd
context vector shape: (8, 200, 2, 16)


In [151]:
np.alltrue(attention_scores_dropout == att_score)

True

In [152]:
np.alltrue(att_output_ == att_output)

True

In [153]:
att_score.shape

TensorShape([8, 2, 200, 200])

In [154]:
np.alltrue(attention_output== model.layers[2].attention_output)

True